In [1]:
import pandas  as pd
import numpy as np

In [3]:
data = pd.read_csv('moviereviews.tsv', sep='\t')

In [4]:
data.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
label     2000 non-null object
review    1965 non-null object
dtypes: object(2)
memory usage: 31.3+ KB


In [7]:
data.shape

(2000, 2)

In [8]:
data['review'][0]

'how do films like mouse hunt get into theatres ? \r\nisn\'t there a law or something ? \r\nthis diabolical load of claptrap from steven speilberg\'s dreamworks studio is hollywood family fare at its deadly worst . \r\nmouse hunt takes the bare threads of a plot and tries to prop it up with overacting and flat-out stupid slapstick that makes comedies like jingle all the way look decent by comparison . \r\nwriter adam rifkin and director gore verbinski are the names chiefly responsible for this swill . \r\nthe plot , for what its worth , concerns two brothers ( nathan lane and an appalling lee evens ) who inherit a poorly run string factory and a seemingly worthless house from their eccentric father . \r\ndeciding to check out the long-abandoned house , they soon learn that it\'s worth a fortune and set about selling it in auction to the highest bidder . \r\nbut battling them at every turn is a very smart mouse , happy with his run-down little abode and wanting it to stay that way . \r\

In [12]:
from IPython.display import Markdown, display
display(Markdown('##### ' + data['review'][0]))

##### how do films like mouse hunt get into theatres ? 
isn't there a law or something ? 
this diabolical load of claptrap from steven speilberg's dreamworks studio is hollywood family fare at its deadly worst . 
mouse hunt takes the bare threads of a plot and tries to prop it up with overacting and flat-out stupid slapstick that makes comedies like jingle all the way look decent by comparison . 
writer adam rifkin and director gore verbinski are the names chiefly responsible for this swill . 
the plot , for what its worth , concerns two brothers ( nathan lane and an appalling lee evens ) who inherit a poorly run string factory and a seemingly worthless house from their eccentric father . 
deciding to check out the long-abandoned house , they soon learn that it's worth a fortune and set about selling it in auction to the highest bidder . 
but battling them at every turn is a very smart mouse , happy with his run-down little abode and wanting it to stay that way . 
the story alternates between unfunny scenes of the brothers bickering over what to do with their inheritance and endless action sequences as the two take on their increasingly determined furry foe . 
whatever promise the film starts with soon deteriorates into boring dialogue , terrible overacting , and increasingly uninspired slapstick that becomes all sound and fury , signifying nothing . 
the script becomes so unspeakably bad that the best line poor lee evens can utter after another run in with the rodent is : " i hate that mouse " . 
oh cringe ! 
this is home alone all over again , and ten times worse . 
one touching scene early on is worth mentioning . 
we follow the mouse through a maze of walls and pipes until he arrives at his makeshift abode somewhere in a wall . 
he jumps into a tiny bed , pulls up a makeshift sheet and snuggles up to sleep , seemingly happy and just wanting to be left alone . 
it's a magical little moment in an otherwise soulless film . 
a message to speilberg : if you want dreamworks to be associated with some kind of artistic credibility , then either give all concerned in mouse hunt a swift kick up the arse or hire yourself some decent writers and directors . 
this kind of rubbish will just not do at all . 


In [13]:
data.describe()

,label,review
count,2000,1965
unique,2,1939
top,neg,
freq,1000,27


In [15]:
## Check for null values
data.isnull().sum()

label      0
review    35
dtype: int64

In [16]:
data.dropna(inplace=True)

In [17]:
data.describe()

,label,review
count,1965,1965
unique,2,1939
top,neg,
freq,983,27


In [19]:
data.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [25]:
blanks = []

for i,lb,rv in data.itertuples(): #iterating the data frame
    if type(rv)==str:  #avoiding NaN values
        if rv.isspace(): 
            blanks.append(i)
            
print(blanks)
len(blanks)

[]


0

In [22]:
data.drop(blanks, inplace=True)

In [23]:
data.shape

(1938, 2)

In [26]:
data['label'].value_counts()

neg    969
pos    969
Name: label, dtype: int64

#### Splitting the data into test train data

In [35]:
from sklearn.model_selection import train_test_split

x = data['review']
y=data['label']

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=42)

# Building pipeline using sklearn packages

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())]) 

text_clf_svc = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())]) 

# Feed training data to first pipeline - Naive Bayes

In [38]:
text_clf_nb.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

# Run predictions and analyze the results of Naive Bayes

In [39]:
predictions = text_clf_nb.predict(x_test)

In [42]:
#import confusion matrcs

from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))
print('\n\n\n')
print(metrics.classification_report(y_test, predictions))

[[259  23]
 [102 198]]




              precision    recall  f1-score   support

         neg       0.72      0.92      0.81       282
         pos       0.90      0.66      0.76       300

   micro avg       0.79      0.79      0.79       582
   macro avg       0.81      0.79      0.78       582
weighted avg       0.81      0.79      0.78       582



In [43]:
print(metrics.accuracy_score(y_test,predictions))

0.7852233676975945


# Feed the data through the second pipeline

In [44]:
text_clf_svc.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

# Running predictions and analyzing the results of linearSVC

In [45]:
predictions = text_clf_svc.predict(x_test)

In [46]:
print(metrics.confusion_matrix(y_test,predictions))
print('\n\n\n')
print(metrics.classification_report(y_test, predictions))

[[235  47]
 [ 41 259]]




              precision    recall  f1-score   support

         neg       0.85      0.83      0.84       282
         pos       0.85      0.86      0.85       300

   micro avg       0.85      0.85      0.85       582
   macro avg       0.85      0.85      0.85       582
weighted avg       0.85      0.85      0.85       582



In [47]:
print(metrics.accuracy_score(y_test,predictions))

0.8487972508591065


# Remove stopwords

In [48]:
# Sk learn comes with its own stopwords list

from sklearn.feature_extraction import text
print(len(text.ENGLISH_STOP_WORDS))

318


In [56]:
swords = text.ENGLISH_STOP_WORDS

In [50]:
df = pd.read_csv('moviereviews.tsv', sep='\t')

In [51]:
df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [54]:
df.dropna(inplace=True)
blanks = []

blanks = []

for i,lb,rv in df.itertuples(): #iterating the data frame
    if type(rv)==str:  #avoiding NaN values
        if rv.isspace(): 
            blanks.append(i)
            
print(blanks)
len(blanks)

[57, 71, 147, 151, 283, 307, 313, 323, 343, 351, 427, 501, 633, 675, 815, 851, 977, 1079, 1299, 1455, 1493, 1525, 1531, 1763, 1851, 1905, 1993]


27

In [55]:
df.drop(blanks, inplace=True)

In [63]:
text_clf_svc2 = Pipeline([('tfidf', TfidfVectorizer(stop_words=swords)), ('clf', LinearSVC())])

In [64]:
text_clf_svc2.fit(x_train,y_train)
predictions3 = text_clf_svc2.predict(x_test)

In [ ]:
myreview